In [21]:
import numpy as np
import pandas as pd
import catboost
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
import json
import matplotlib.pyplot as plt

In [22]:
df = pd.read_csv('Important.csv')
df.shape

(335113, 14)

In [23]:
df.head()

,Unnamed: 0,s_247_71,s_247_73,s_247_75,s_247_77,s_247_78,s_248_70,s_248_72,s_248_74,s_248_76,OfferID,indiv_id,target,probability
0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,78,1,0,0.000935
1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,78,10,0,0.005847
2,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,78,22,0,0.001941
3,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,78,28,0,0.002613
4,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,78,61,0,0.001315


In [24]:
OfferID_list=[642,640,646,645,88]

In [25]:
df = df[df['OfferID'].isin(OfferID_list)]
df.shape

(199106, 14)

In [26]:
df.columns

Index(['Unnamed: 0', 's_247_71', 's_247_73', 's_247_75', 's_247_77',
       's_247_78', 's_248_70', 's_248_72', 's_248_74', 's_248_76', 'OfferID',
       'indiv_id', 'target', 'probability'],
      dtype='object')

In [27]:
# 'Unnamed: 0', 's_247_71', 's_247_73', 's_247_75', 's_247_77',
#        's_247_78', 's_248_70', 's_248_72', 's_248_74', 's_248_76', 'OfferID',
#        'indiv_id', 'target', 'probability'

In [28]:
df= df[['s_247_71','s_248_70','OfferID',
       'indiv_id', 'target', 'probability']]

In [29]:
df.shape

(199106, 6)

In [30]:
df = df[(df['s_247_71'] == 1) & (df['s_248_70'] == 1)]

In [31]:
df.shape

(1807, 6)

In [32]:
df.head()

,s_247_71,s_248_70,OfferID,indiv_id,target,probability
36551,1.0,1.0,88,97,0,0.004447
36619,1.0,1.0,88,218,0,0.004343
36893,1.0,1.0,88,713,0,0.003400
36927,1.0,1.0,88,775,0,0.004049
36988,1.0,1.0,88,873,0,0.003548


In [33]:
df['OfferID'].value_counts()

OfferID
88     455
640    338
642    338
645    338
646    338
Name: count, dtype: int64

In [34]:
df.groupby(['OfferID'])['probability'].mean()

OfferID
88     0.004344
640    0.709943
642    0.708206
645    0.708129
646    0.711075
Name: probability, dtype: float64

In [35]:
# ranking - 646(0.711075), 640(0.709943),642(0.708206), 645(0.708129), 88(0.004344)

In [36]:
# ranking - 640 (0.27) , 642(0.25) , 645(0.23) , 646 (0.22) , 88 (0)

In [37]:
accp_prob_rank = [640 , 642, 645, 646, 88]

## Analysis Two

In [38]:
df.head()

,s_247_71,s_248_70,OfferID,indiv_id,target,probability
36551,1.0,1.0,88,97,0,0.004447
36619,1.0,1.0,88,218,0,0.004343
36893,1.0,1.0,88,713,0,0.003400
36927,1.0,1.0,88,775,0,0.004049
36988,1.0,1.0,88,873,0,0.003548


In [39]:
df.nunique()

s_247_71         1
s_248_70         1
OfferID          5
indiv_id       542
target           2
probability    108
dtype: int64

In [40]:
df['indiv_id'].value_counts()

indiv_id
415132288     5
415349726     5
415777647     5
1501626801    5
1505388520    5
             ..
58720         1
58815         1
58871         1
59535         1
55005         1
Name: count, Length: 542, dtype: int64

In [58]:
# Get the value counts of indiv_id
value_counts = df['indiv_id'].value_counts()

# Filter the value counts to include only those with a count of 5
indiv_ids_with_count_5 = value_counts[value_counts == 5].index

# Display the individual IDs with a value count of 5
indiv_ids_with_count_5



Index([ 415132288,  415349726,  415777647, 1501626801, 1505388520,       9634,
             8359,       8714,       9255,       9412,
       ...
             2669,       3479,       3543,       3749,       3837,        985,
               97,        218,        912,        926],
      dtype='int64', name='indiv_id', length=251)

In [51]:
top_indiv_id = list(df['indiv_id'].value_counts().head(10).index)

In [52]:
top_indiv_id

[415132288,
 415349726,
 415777647,
 1501626801,
 1505388520,
 9634,
 8359,
 8714,
 9255,
 9412]

In [43]:
temp = df[df['indiv_id'] == 97]

In [44]:
temp

,s_247_71,s_248_70,OfferID,indiv_id,target,probability
36551,1.0,1.0,88,97,0,0.004447
158414,1.0,1.0,640,97,0,0.711890
199064,1.0,1.0,642,97,1,0.709691
239714,1.0,1.0,645,97,0,0.709559
280364,1.0,1.0,646,97,1,0.712204


In [45]:
# Sorting by column
temp= temp.sort_values(by=['probability'], ascending=False)

In [46]:
temp

,s_247_71,s_248_70,OfferID,indiv_id,target,probability
280364,1.0,1.0,646,97,1,0.712204
158414,1.0,1.0,640,97,0,0.711890
199064,1.0,1.0,642,97,1,0.709691
239714,1.0,1.0,645,97,0,0.709559
36551,1.0,1.0,88,97,0,0.004447


In [47]:
list(temp['OfferID'])

[646, 640, 642, 645, 88]

In [48]:
accp_prob_rank

[640, 642, 645, 646, 88]

In [49]:
for indiv_id in top_indiv_id:
    temp = df[df['indiv_id'] == indiv_id]
    temp = temp.sort_values(by=['probability'], ascending=False)
    offer_ids = list(temp['OfferID'])
    print(f"Top Offers for indiv_id {indiv_id}: {offer_ids}")

Top Offers for indiv_id 415132288: [646, 640, 642, 645, 88]
Top Offers for indiv_id 415349726: [646, 640, 642, 645, 88]
Top Offers for indiv_id 415777647: [646, 640, 645, 642, 88]
Top Offers for indiv_id 1501626801: [646, 640, 642, 645, 88]
Top Offers for indiv_id 1505388520: [640, 645, 646, 642, 88]
Top Offers for indiv_id 9634: [646, 640, 645, 642, 88]
Top Offers for indiv_id 8359: [646, 640, 642, 645, 88]
Top Offers for indiv_id 8714: [646, 640, 642, 645, 88]
Top Offers for indiv_id 9255: [646, 640, 645, 642, 88]
Top Offers for indiv_id 9412: [646, 640, 642, 645, 88]


In [50]:
accp_prob_rank

[640, 642, 645, 646, 88]